In [ ]:
# | default_exp _components.encoder.avro

In [ ]:
# | export


import io
from typing import *

import fastavro
from pydantic import BaseModel
from pydantic.main import ModelMetaclass

from fastkafka._components.logger import get_logger

In [ ]:
import json
import tempfile

from pydantic import Field

from fastkafka._components.logger import supress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


class AvroBase(BaseModel):
    """This is base pydantic class that will add some methods"""

    @classmethod
    def avro_schema_for_pydantic(
        cls,
        pydantic_model: Union[BaseModel, ModelMetaclass],
        by_alias: bool = True,
        namespace: Optional[str] = None,
    ) -> Dict[str, Any]:
        """
        Return the avro schema for the pydantic class

        :param by_alias: generate the schemas using the aliases defined, if any
        :param namespace: Provide an optional namespace string to use in schema generation
        :return: dict with the Avro Schema for the model
        """

        if isinstance(pydantic_model, BaseModel):
            schema = pydantic_model.__class__.schema(by_alias=by_alias)
        elif isinstance(pydantic_model, ModelMetaclass):
            schema = pydantic_model.schema(by_alias=by_alias)
        else:
            raise ValueError(
                f"Unknown type {type(pydantic_model)} given for pydantic_model parameter"
            )

        if namespace is None:
            # default namespace will be based on title
            namespace = schema["title"]

        return cls._avro_schema(schema, namespace)

    @classmethod
    def avro_schema(
        cls, by_alias: bool = True, namespace: Optional[str] = None
    ) -> Dict[str, Any]:
        """
        Return the avro schema for the pydantic class

        :param by_alias: generate the schemas using the aliases defined, if any
        :param namespace: Provide an optional namespace string to use in schema generation
        :return: dict with the Avro Schema for the model
        """
        schema = cls.schema(by_alias=by_alias)

        if namespace is None:
            # default namespace will be based on title
            namespace = schema["title"]

        return cls._avro_schema(schema, namespace)

    @staticmethod
    def _avro_schema(schema: Dict[str, Any], namespace: str) -> Dict[str, Any]:
        """Return the avro schema for the given pydantic schema"""

        classes_seen = set()

        def get_definition(ref: str, schema: Dict[str, Any]) -> Dict[str, Any]:
            """Reading definition of base schema for nested structs"""
            id = ref.replace("#/definitions/", "")
            d = schema.get("definitions", {}).get(id)
            if d is None:
                raise RuntimeError(f"Definition {id} does not exist")
            return d  # type: ignore

        def get_type(value: Dict[str, Any]) -> Dict[str, Any]:
            """Returns a type of a single field"""
            t = value.get("type")
            f = value.get("format")
            r = value.get("$ref")
            a = value.get("additionalProperties")
            avro_type_dict: Dict[str, Any] = {}
            if "default" in value:
                avro_type_dict["default"] = value.get("default")
            if "description" in value:
                avro_type_dict["doc"] = value.get("description")
            if "allOf" in value and len(value["allOf"]) == 1:
                r = value["allOf"][0]["$ref"]
            if r is not None:
                class_name = r.replace("#/definitions/", "")
                if class_name in classes_seen:
                    avro_type_dict["type"] = class_name
                else:
                    d = get_definition(r, schema)
                    if "enum" in d:
                        avro_type_dict["type"] = {
                            "type": "enum",
                            "symbols": [str(v) for v in d["enum"]],
                            "name": d["title"],
                        }
                    else:
                        avro_type_dict["type"] = {
                            "type": "record",
                            "fields": get_fields(d),
                            # Name of the struct should be unique true the complete schema
                            # Because of this the path in the schema is tracked and used as name for a nested struct/array
                            "name": class_name,
                        }
                    classes_seen.add(class_name)
            elif t == "array":
                items = value.get("items")
                tn = get_type(items)  # type: ignore
                # If items in array are a object:
                if "$ref" in items:  # type: ignore
                    tn = tn["type"]
                # If items in array are a logicalType
                if (
                    isinstance(tn, dict)
                    and isinstance(tn.get("type", {}), dict)
                    and tn.get("type", {}).get("logicalType") is not None
                ):
                    tn = tn["type"]
                avro_type_dict["type"] = {"type": "array", "items": tn}
            elif t == "string" and f == "date-time":
                avro_type_dict["type"] = {
                    "type": "long",
                    "logicalType": "timestamp-micros",
                }
            elif t == "string" and f == "date":
                avro_type_dict["type"] = {
                    "type": "int",
                    "logicalType": "date",
                }
            elif t == "string" and f == "time":
                avro_type_dict["type"] = {
                    "type": "long",
                    "logicalType": "time-micros",
                }
            elif t == "string" and f == "uuid":
                avro_type_dict["type"] = {
                    "type": "string",
                    "logicalType": "uuid",
                }
            elif t == "string":
                avro_type_dict["type"] = "string"
            elif t == "number":
                avro_type_dict["type"] = "double"
            elif t == "integer":
                # integer in python can be a long
                avro_type_dict["type"] = "long"
            elif t == "boolean":
                avro_type_dict["type"] = "boolean"
            elif t == "object":
                if a is None:
                    value_type = "string"
                else:
                    value_type = get_type(a)  # type: ignore
                if isinstance(value_type, dict) and len(value_type) == 1:
                    value_type = value_type.get("type")  # type: ignore
                avro_type_dict["type"] = {"type": "map", "values": value_type}
            else:
                raise NotImplementedError(
                    f"Type '{t}' not support yet, "
                    f"please report this at https://github.com/godatadriven/pydantic-avro/issues"
                )
            return avro_type_dict

        def get_fields(s: Dict[str, Any]) -> List[Dict[str, Any]]:
            """Return a list of fields of a struct"""
            fields = []

            required = s.get("required", [])
            for key, value in s.get("properties", {}).items():
                avro_type_dict = get_type(value)
                avro_type_dict["name"] = key

                if key not in required:
                    if avro_type_dict.get("default") is None:
                        avro_type_dict["type"] = ["null", avro_type_dict["type"]]
                        avro_type_dict["default"] = None

                fields.append(avro_type_dict)
            return fields

        fields = get_fields(schema)

        return {
            "type": "record",
            "namespace": namespace,
            "name": schema["title"],
            "fields": fields,
        }

In [ ]:
test_user_schema = {
    "type": "record",
    "namespace": "User",
    "name": "User",
    "fields": [
        {"type": "string", "name": "name"},
        {"type": ["null", "long"], "name": "favorite_number", "default": None},
        {"type": ["null", "string"], "name": "favorite_color", "default": None},
    ],
}


class User(BaseModel):
    name: str
    favorite_number: Optional[int] = None
    favorite_color: Optional[str] = None

In [ ]:
actual = AvroBase.avro_schema_for_pydantic(User)
display(actual)
assert actual == test_user_schema

{'type': 'record',
 'namespace': 'User',
 'name': 'User',
 'fields': [{'type': 'string', 'name': 'name'},
  {'type': ['null', 'long'], 'name': 'favorite_number', 'default': None},
  {'type': ['null', 'string'], 'name': 'favorite_color', 'default': None}]}

In [ ]:
# ToDo
# 1. Rewrite with fastavro - Done
# 2. Generate schema from pydantic itself - Done
#        - Pydantic to avro schema conversion methods - Done
# 3. Generate pydantic class from avro schema

In [ ]:
# | export


def avro_encoder(msg: BaseModel) -> bytes:
    schema = fastavro.schema.parse_schema(AvroBase.avro_schema_for_pydantic(msg))
    bytes_writer = io.BytesIO()
    fastavro.schemaless_writer(bytes_writer, schema, msg.dict())
    raw_bytes = bytes_writer.getvalue()
    return raw_bytes

In [ ]:
msg = User(name="Kumaran", favorite_number=9, favorite_color="black")


actual = avro_encoder(msg)
display(actual)

assert isinstance(actual, bytes)
assert actual == b"\x0eKumaran\x02\x12\x02\nblack"

b'\x0eKumaran\x02\x12\x02\nblack'

In [ ]:
# | export


def avro_decoder(raw_msg: bytes, cls: ModelMetaclass) -> Any:
    schema = fastavro.schema.parse_schema(AvroBase.avro_schema_for_pydantic(cls))

    bytes_reader = io.BytesIO(raw_msg)
    msg_dict = fastavro.schemaless_reader(bytes_reader, schema)

    return cls(**msg_dict)

In [ ]:
raw_msg = b"\x06123\x02\x00\x02\x06111"


actual = avro_decoder(raw_msg, cls=User)
display(actual)

assert isinstance(actual, User)
assert actual.name == "123"
assert actual.favorite_number == 0
assert actual.favorite_color == "111"

User(name='123', favorite_number=0, favorite_color='111')